## Рекомендация статей с помощью тематических моделей

In [1]:
from scipy.special import softmax
from collections import OrderedDict
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from collections import Counter
from sklearn import metrics
import numpy as np
import string
import tqdm

In [ ]:
arxiv_tokens = OrderedDict()
stop_words = set(stopwords.words('english'))
with open('data/arxiv_plain.txt', 'r') as f:
    for line in tqdm.tqdm(f):
        cur_tokens = line.split()
        arxiv_tokens[cur_tokens[0]] = list(filter(lambda token: token not in stop_words, cur_tokens[1:]))
arxiv_titles = list(arxiv_tokens.keys())

In [ ]:
arxiv_dictionary = Dictionary(list(arxiv_tokens.values()))
arxiv_corpus = [arxiv_dictionary.doc2bow(text) for text in list(arxiv_tokens.values())]

In [ ]:
lda = LdaModel(arxiv_corpus, num_topics=300)

In [ ]:
lda.num_topics

In [ ]:
lda.get_document_topics(arxiv_corpus[0])

In [ ]:
theta = {}
for doc_title, doc_bow in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = topic_vector

Тематический вектор статьи с номером 0704.0004:

In [ ]:
theta['0704.0004']

Теперь для того, чтобы порекомендовать читателю близкие по смыслу статьи, достаточно выбрать метрику близости и сравнить вектор текущего документа (например, последнего прочитанного) с векторами всех остальных документов в коллекции. В качестве метрики близости можно использовать косинусную меру, евклидово расстояние, расстояние Хелингера и т.д.

In [9]:
def cos_sim(first, second):
    return metrics.pairwise.cosine_similarity(first.reshape(1, -1), second.reshape(1, -1))[0][0]

def dot_sim(first, second):
    return first.dot(second)

def hel_sim(first, second): #one more sqrt and division by sqrt(2) omitted, minus added
    return -np.sum((np.sqrt(first) - np.sqrt(second)) ** 2)

def jaccard_sim(first, second):
    intersection = set(first).intersection(set(second))
    union = set(first).union(set(second))
    return float(len(intersection))/float(len(union))

In [ ]:
def recommend_papers(query, theta, sim=cos_sim, top_k=10):
    query_vec = theta[query]
    ranked_list = []
    for doc_name, doc_vec in theta.items():
        ranked_list.append((doc_name, sim(query_vec, doc_vec)))
    ranked_list.sort(key=lambda x: x[1], reverse=True)
    return ranked_list[:top_k]

In [ ]:
recommended_papers = recommend_papers('0704.2596', theta, top_k=5)

In [ ]:
for paper_name, prob in recommended_papers:
    print(paper_name)
    print(' '.join([token[0] for token in Counter(arxiv_tokens[paper_name]).most_common(10)]))
    print()

Для оценки качества полученной рекомендательной системы воспользуемся датасетом триплетов [[Dai et al. 2015](https://arxiv.org/abs/1507.07998)]. Датасет содержит тройки статей `<запрос>|<релевантная статья>|<нерелевантная статья>`. Будем считать, что если метрика близости между запросом и релевантной статьей оказалась выше, чем между запросом и нерелевантной статьей, то такая тройка обработана "правильно".

In [22]:
def evaluate_quality(theta, sim):
    all_triplets = 0
    covered_triplets = 0
    correct_triplets = 0
    with open('data/arxiv_triplets.txt', 'r') as fin:
        for line in fin:
            ids = list(map(lambda x: x.split('/pdf/')[-1], line.split()))
            if all([x in theta.keys() for x in ids]):
                covered_triplets += 1
                vectors = [theta[x] for x in ids]
                correct_triplets += sim(vectors[0], vectors[1]) > sim(vectors[0], vectors[2])
            all_triplets += 1
    return 1.0 * correct_triplets / covered_triplets

In [ ]:
evaluate_quality(theta, cos_sim)

In [ ]:
evaluate_quality(theta, hel_sim)

In [ ]:
evaluate_quality(theta, dot_sim)

Попробуем 300 тем

In [ ]:
lda = LdaModel(arxiv_corpus, num_topics=300)

theta = {}
for doc_title, doc_bow in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = topic_vector
    
print(evaluate_quality(theta, cos_sim))

### Эксперимент №2: использовать BERT-based фичи совместно с тематическими фичами

In [2]:
from transformers import BertTokenizer, BertModel, BertTokenizerFast
from gensim.matutils import Sparse2Corpus
from scipy import sparse
from nltk import word_tokenize
import json
import torch
import pickle 

BERT: http://jalammar.github.io/illustrated-bert/

Loading abtracts

In [3]:
with open('data/arxiv_metadata.json', 'r') as f:
    data = f.read().split('\n')

In [4]:
arxiv_titles = []
with open('data/arxiv_plain.txt', 'r') as f:
    for line in tqdm.tqdm(f):
        cur_tokens = line.split()
        arxiv_titles.append(cur_tokens[0])
arxiv_titles = set(arxiv_titles)

43091it [00:14, 2991.13it/s]


In [5]:
arxiv_tokens = {}
stop_words = set(stopwords.words('english'))
for article in tqdm.tqdm(data[:-1]):
    arxiv_json =  json.loads(article)
    if arxiv_json['id'] not in arxiv_titles:
        continue
    
    text = arxiv_json['abstract'].lower()
    text_p = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text_p)
    filtered_words = [word for word in words if word not in stop_words]
    
    arxiv_tokens[arxiv_json['id']] = filtered_words

100%|██████████| 1796911/1796911 [01:01<00:00, 29196.28it/s]


In [26]:
arxiv_titles = list(arxiv_tokens.keys())

Loading bert representations

In [6]:
with open('bert_repr_4l.pickle', 'rb') as f:
    bert_reprs =  pickle.load(f)

#### Bert quality

In [48]:
theta_bert = {}
for doc_title, bert_vect in tqdm.tqdm(zip(arxiv_titles,  bert_reprs)):
    theta_bert[doc_title] = bert_vect
evaluate_quality(theta_bert, dot_sim)

43091it [00:00, 974332.63it/s]


0.0

In [28]:
theta_bert_softmax = {}
for doc_title, bert_vect in tqdm.tqdm(zip(arxiv_titles,  bert_reprs)):
    theta_bert_softmax[doc_title] = softmax(bert_vect)
evaluate_quality(theta_bert_softmax, cos_sim)

43091it [00:02, 20087.23it/s]


0.0

#### Concatenating Bert and LDA

In [29]:
arxiv_dictionary = Dictionary(list(arxiv_tokens.values()))
arxiv_corpus = [arxiv_dictionary.doc2bow(text) for text in list(arxiv_tokens.values())]
lda = LdaModel(arxiv_corpus, num_topics=300)

LDA

In [30]:
theta = {}
for doc_title, doc_bow in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = topic_vector
evaluate_quality(theta, cos_sim)

43091it [01:00, 708.66it/s]


0.6953888853844699

LDA + BERT

In [31]:
theta = {}
for doc_title, doc_bow, bert_vect in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus, bert_reprs)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = np.concatenate((topic_vector, bert_vect))
evaluate_quality(theta, cos_sim)

43091it [01:04, 667.21it/s]


0.6178010471204188

LDA + softmax(Bert)

In [32]:
theta = {}
for doc_title, doc_bow, bert_vect in tqdm.tqdm(zip(arxiv_titles, arxiv_corpus, bert_reprs)):
    topic_vector = np.zeros(lda.num_topics)
    for topic_num, topic_prob in lda.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = np.concatenate((topic_vector, softmax(bert_vect)))
evaluate_quality(theta, cos_sim)

43091it [01:05, 660.86it/s]


0.7045984987068694

#### LDA on Bert

In [33]:
bert_sparse = sparse.csr_matrix(np.array(bert_reprs).T)
corpus = Sparse2Corpus(bert_sparse)
lda_bert = LdaModel(corpus, num_topics=300, chunksize = 50000, minimum_probability=0.0)

/Users/ayazhan/anaconda3/lib/python3.8/site-packages/gensim/models/ldamodel.py:695: RuntimeWarning: overflow encountered in exp
  expElogthetad = np.exp(Elogthetad)
/Users/ayazhan/anaconda3/lib/python3.8/site-packages/gensim/models/ldamodel.py:693: RuntimeWarning: invalid value encountered in multiply
  gammad = self.alpha + expElogthetad * np.dot(cts / phinorm, expElogbetad.T)


In [37]:
lda_bert.get_document_topics(corpus[0])

[]

In [42]:
bert_sparse = sparse.csr_matrix(softmax(np.array(bert_reprs), axis=1).T)
corpus = Sparse2Corpus(bert_sparse)
lda_bert = LdaModel(corpus, num_topics=300, chunksize = 50000, minimum_probability=0.0)

In [44]:
theta = {}
for doc_title, doc_bow in tqdm.tqdm(zip(arxiv_titles, corpus)):
    topic_vector = np.zeros(lda_bert.num_topics)
    for topic_num, topic_prob in lda_bert.get_document_topics(doc_bow):
        topic_vector[topic_num] = topic_prob
    theta[doc_title] = topic_vector

43091it [01:19, 544.39it/s]


In [45]:
evaluate_quality(theta, cos_sim)

0.0

### Fasttext